In [1]:
!pip install pyspark==3.5.5

In [1]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import shutil
import kagglehub
import torch
from pyspark.sql import SparkSession
import socket

dataset_path = os.path.join(ROOT_DIR, 'neural', 'datasets', 'spark', 'test_1')
weight_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_5', 'kmeans')
os.makedirs(dataset_path, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



print(socket.gethostbyname("spark-master"))
driver_host = socket.gethostbyname(socket.gethostname())
print(driver_host)
driver_host = socket.gethostbyname("spark-master")
print(driver_host)
driver_host = "producer"
os.environ["PYSPARK_SUBMIT_ARGS"] = f"""
--master spark://spark-master:7077
--conf spark.driver.host={driver_host}
--conf spark.driver.port=45555
--conf spark.executor.memory=1G
--conf spark.executor.cores=1
pyspark-shell
"""
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'  # путь к Python в контейнере
# os.environ['SPARK_HOME'] = '/opt/spark'            # путь к Spark, укажи реальный
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
neo4j_url  = 'bolt://neo4j:7687'
neo4j_user = os.environ["NEO4J_USER"]
neo4j_pass = os.environ["NEO4J_PASSWORD"]
MONGO_USER = os.environ["MONGO_USER"]
MONGO_PASS = os.environ["MONGO_PASSWORD"]
MONGO_ADDR = f"{MONGO_USER}:{MONGO_PASS}@mongodb:27017"  # :27017

def spark_app_generator(name):
    spark = SparkSession.builder \
        .master("spark://spark-master:7077") \
        .appName(name) \
         .config("spark.driver.bindAddress", "0.0.0.0") \
        .config("spark.executor.memory", "1g") \
        .config("spark.driver.memory", "1g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.3.0,org.neo4j:neo4j-connector-apache-spark_2.12:5.3.1_for_spark_3,com.redislabs:spark-redis_2.12:3.1.0") \
    .config("spark.mongodb.read.connection.uri", f"mongodb://{MONGO_ADDR}") \
    .config("spark.mongodb.write.connection.uri", f"mongodb://{MONGO_ADDR}") \
    .config("spark.redis.host", "redis") \
    .config("spark.redis.port", "6379") \
    .config("spark.redis.db", "0") \
    .config("spark.neo4j.bolt.url", neo4j_url) \
    .config("spark.neo4j.bolt.user", neo4j_user) \
    .config("spark.neo4j.bolt.password", neo4j_pass) \
    .config("neo4j.url", neo4j_url) \
    .config("neo4j.authentication.basic.username", neo4j_user) \
    .config("neo4j.authentication.basic.password", neo4j_pass) \
        .getOrCreate()

        # .config("spark.hadoop.hadoop.home.dir", "/hadoop/dfs/name") \
    # .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    # .config("spark.hadoop.dfs.client.use.namenode.hostname", "true") \
    print(spark.sparkContext.getConf().get("spark.jars.packages"))
    return spark  # /openfoodfacts.products


172.22.0.4
172.22.0.5
172.22.0.4


In [2]:
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType, MapType,
                               DoubleType
                               )

# Пример определения схемы. Настройте схему под структуру ваших данных.
custom_schema = StructType([
    StructField("_id", StringType(), True),
    StructField("product_name", StringType(), True),
    # Если nutriments представляет собой динамические поля, лучше сохранить его как MapType.
    StructField("ingredients_n", IntegerType(), nullable = True),
      StructField("ingredients_sweeteners_n", IntegerType(), nullable = True),
      StructField("scans_n", IntegerType(), nullable = True),
      StructField("additives_n", IntegerType(), nullable = True),


])

spark = spark_app_generator('lab_7')
df = spark.read.format("org.neo4j.spark.DataSource") \
    .option("labels", "CleanedProduct") \
    .load()    # , database="off", collection='products'

# Просмотр схемы и первых строк
print(df.count())
df.printSchema()
df.show(20)


:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
com.redislabs#spark-redis_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1df7e46a-3032-4ae5-b5fc-8df227feec1f;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.3.0 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12;5.3.1_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.3.1 in central
	found org.neo4j.driver#neo4j-java-driver;4.4.17 in central
	found org.reactivestreams#reactive-streams

org.mongodb.spark:mongo-spark-connector_2.12:10.3.0,org.neo4j:neo4j-connector-apache-spark_2.12:5.3.1_for_spark_3,com.redislabs:spark-redis_2.12:3.1.0


25/04/26 20:03:25 WARN SchemaService: Switching to query schema resolution
                                                                                

500435
root
 |-- <id>: long (nullable = false)
 |-- <labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- scans_n: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- ingredients_sweeteners_n: long (nullable = true)
 |-- ingredients_n: long (nullable = true)
 |-- additives_n: long (nullable = true)
 |-- _id: string (nullable = true)



+----+----------------+-------+--------------------+------------------------+-------------+-----------+-------------+
|<id>|        <labels>|scans_n|        product_name|ingredients_sweeteners_n|ingredients_n|additives_n|          _id|
+----+----------------+-------+--------------------+------------------------+-------------+-----------+-------------+
|   0|[CleanedProduct]|      1|      Genovese pesto|                       0|           17|          0|0011110899378|
|   1|[CleanedProduct]|      2|        Marshmallows|                       0|            9|          3|0015400023931|
|   2|[CleanedProduct]|      3|    Meatless Chorizo|                       0|           20|          1|0011110129345|
|   3|[CleanedProduct]|      2|                Cola|                       0|            6|          2|0011110496805|
|   4|[CleanedProduct]|      1|Wrap Poulet à la ...|                       0|           51|          5|     00099332|
|   5|[CleanedProduct]|      1|        Berry Medley|    

In [3]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Сбор признаков в вектор
numeric_features_2 = ['ingredients_n', 'ingredients_sweeteners_n', 'scans_n', 'additives_n',]
assembler = VectorAssembler(inputCols=numeric_features_2, outputCol="features_assembled")
df_vector = assembler.transform(df)

# Масштабирование признаков
scaler = StandardScaler(inputCol="features_assembled", outputCol="features")
scaler_model = scaler.fit(df_vector)
df_scaled = scaler_model.transform(df_vector)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


# df_small = df_scaled.limit(100)
# Обучение модели k-средних
kmeans = KMeans().setK(4).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(df_scaled)



[Stage 10:>                                                         (0 + 1) / 1]

In [6]:
numeric_features_2

['ingredients_n', 'ingredients_sweeteners_n', 'scans_n', 'additives_n']

In [ ]:
# Предсказание кластеров
predictions = model.transform(df_scaled)
result = predictions.select(
    "_id",
    "product_name",
    'ingredients_n', 'ingredients_sweeteners_n', 'scans_n', 'additives_n',
    "features", "prediction"
)
# result.show()
# Оценка модели
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette}")

# Центры кластеров
centers = model.clusterCenters()
for idx, center in enumerate(centers):
    print(f"Cluster {idx} center: {center}")

In [ ]:
result.show(10)

In [ ]:

result \
 .write\
 .format("org.apache.spark.sql.redis")\
 .option("table", "results_lab_7")\
 .option("key.column", "_id")\
 .mode("overwrite")\
 .save()

In [12]:

# weight_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_5', 'kmeans_3')
# os.makedirs(weight_path, exist_ok=True)
# model.save("/opt/bitnami/spark/work/labs/lab_5/kmeans_5")
model.save('/opt/bitnami/spark/work/labs/lab_5/kmeans_7')

In [6]:
# Не забудьте остановить SparkSession по завершении работы
spark.stop()

In [13]:
from pyspark.ml.clustering import KMeansModel

loaded_model = KMeansModel.load("/opt/bitnami/spark/work/labs/lab_5/kmeans_7")


UnsupportedOperationException: empty collection

In [12]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("KMeansExample") \
    .getOrCreate()

# Пример данных
data = [
    (0.0, 0.0),
    (1.0, 1.0),
    (9.0, 8.0),
    (8.0, 9.0)
]
columns = ["x", "y"]
df = spark.createDataFrame(data, columns)

# Преобразование признаков в вектор
assembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
dataset = assembler.transform(df)

# Обучение модели KMeans
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Сохранение модели
model.save("/opt/bitnami/spark/work/labs/lab_5/kmeans_test")

# Загрузка модели
from pyspark.ml.clustering import KMeansModel
loaded_model = KMeansModel.load("/opt/bitnami/spark/work/labs/lab_5/kmeans_test")

# Применение загруженной модели
predictions = loaded_model.transform(dataset)
predictions.show()


25/04/22 14:25:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


UnsupportedOperationException: empty collection